<a href="https://colab.research.google.com/github/CharlesPrado23/BCDataScience1T22N/blob/main/Sistema_Recomendacao/Collaborative_aula(NearestNeighbors_UserBased).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

# Collaborative Filtering

Estimated time needed: **25** minutes

## Objectives

After completing this lab you will be able to:

-   Create recommendation system based on collaborative filtering


Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous can be commonly seen in online stores, movies databases and job finders. In this notebook, we will explore recommendation systems based on Collaborative Filtering and implement simple version of one using Python and the Pandas library.


<h1>Table of contents</h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#ref1">Acquiring the Data</a></li>
        <li><a href="#ref2">Preprocessing</a></li>
        <li><a href="#ref3">Collaborative Filtering</a></li>
    </ol>
</div>
<br>
<hr>


<a id="ref1"></a>

# Acquiring the Data


To acquire and extract the data, simply run the following Bash scripts:  
Dataset acquired from [GroupLens](http://grouplens.org/datasets/movielens?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork-20718538&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork-20718538&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork-20718538&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork-20718538&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ). Lets download the dataset. To download the data, we will use **`!wget`** to download it from IBM Object Storage.  
**Did you know?** When it comes to Machine Learning, you will likely be working with large datasets. As a business, where can you host your data? IBM is offering a unique opportunity for businesses, with 10 Tb of IBM Cloud Object Storage: [Sign up now for free](http://cocl.us/ML0101EN-IBM-Offer-CC)


In [1]:
!wget -O moviedataset.zip https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/data/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 

--2022-05-25 21:19:30--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/data/moviedataset.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160301210 (153M) [application/zip]
Saving to: ‘moviedataset.zip’

moviedataset.zip    100%[===================>] 152.88M  40.9MB/s    in 3.7s    

2022-05-25 21:19:34 (40.9 MB/s) - ‘moviedataset.zip’ saved [160301210/160301210]

unziping ...
Archive:  moviedataset.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: tags.csv        

Now you're ready to start working with the data!


<hr>

<a id="ref2"></a>

# Preprocessing


First, let's get all of the imports out of the way:


In [2]:
#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Now let's read each file into their Dataframes:


In [3]:
#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('movies.csv',usecols=['movieId','title'])
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('ratings.csv',usecols=['userId','movieId','rating'])

Let's also take a peek at how each of them are organized:


In [4]:
#Head is a function that gets the first N rows of a dataframe. N's default is 5.
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


<br>


Next, let's look at the ratings dataframe.


In [5]:
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


In [6]:
res = movies_df[movies_df.title.str.contains('Akira')]
res

,movieId,title
1246,1274,Akira (1988)
6172,6270,Akira Kurosawa's Dreams (Dreams) (1990)
24106,113634,Message from Akira Kurosawa: For Beautiful Mov...


<hr>

<a id="ref3"></a>

# Collaborative Filtering


Now, time to start our work on recommendation systems. 

The first technique we're going to take a look at is called **Collaborative Filtering**, which is also known as **User-User Filtering**. As hinted by its alternate name, this technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. There are several methods of finding similar users (Even some making use of Machine Learning), and the one we will be using here is going to be based on the **Pearson Correlation Function**.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/images/User_Item.png" width=800px>

The process for creating a User Based recommendation system is as follows:

-   Select a user with the movies the user has watched
-   Based on his rating to movies, find the top X neighbours 
-   Get the watched movie record of the user for each neighbour.
-   Calculate a similarity score using some formula
-   Recommend the items with the highest score

Let's begin by creating an input user to recommend movies to:

Notice: To add more movies, simply increase the amount of elements in the userInput. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .


In [7]:
userInput = [
            {'title':'Breakfast Club, The (1985)', 'rating':1.5},
            {'title':'Toy Story (1995)', 'rating':3.5},
            {'title':'Jumanji (1995)', 'rating':4.5},
            {'title':"Pulp Fiction (1994)", 'rating':2.5},
            {'title':'Akira (1988)', 'rating':5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The (1985)",1.5
1,Toy Story (1995),3.5
2,Jumanji (1995),4.5
3,Pulp Fiction (1994),2.5
4,Akira (1988),5.0


#### Add movieId to input user

With the input complete, let's extract the input movies's ID's from the movies dataframe and add them into it.

We can achieve this by first filtering out the rows that contain the input movies' title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.


In [8]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe

#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

,movieId,title,rating
0,1,Toy Story (1995),3.5
1,2,Jumanji (1995),4.5
2,296,Pulp Fiction (1994),2.5
3,1274,Akira (1988),5.0
4,1968,"Breakfast Club, The (1985)",1.5


#### The users who has seen the same movies

Now with the movie ID's in our input, we can now get the subset of users that have watched and reviewed the movies in our input.


In [9]:
#Filtering out users that have watched movies that the input has watched and storing it
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
#userSubset = userSubset.sort_values(by=['rating'], ascending=False)

userSubset

,userId,movieId,rating
19,4,296,4.0
441,12,1968,3.0
479,13,2,2.0
531,13,1274,5.0
681,14,296,2.0
...,...,...,...
22883679,247738,296,4.0
22884132,247751,1,4.0
22884142,247751,296,4.0
22884164,247751,1274,5.0


In [10]:
userSubset.isna().sum()
len(userSubset)

196623

In [11]:
from scipy.sparse import csr_matrix
movies_users = userSubset.pivot_table(index = 'userId', columns='movieId',values='rating')
movies_users



movieId,1,2,296,1274,1968
userId,,,,,
4,NaN,NaN,4.0,NaN,NaN
12,NaN,NaN,NaN,NaN,3.0
13,NaN,2.0,NaN,5.0,NaN
14,NaN,NaN,2.0,NaN,NaN
15,4.0,NaN,3.0,NaN,3.0
...,...,...,...,...,...
247734,NaN,NaN,NaN,5.0,NaN
247735,4.5,NaN,NaN,NaN,NaN
247738,NaN,NaN,4.0,NaN,NaN


In [12]:
movies_users = movies_users.fillna(0)
movies_users

movieId,1,2,296,1274,1968
userId,,,,,
4,0.0,0.0,4.0,0.0,0.0
12,0.0,0.0,0.0,0.0,3.0
13,0.0,2.0,0.0,5.0,0.0
14,0.0,0.0,2.0,0.0,0.0
15,4.0,0.0,3.0,0.0,3.0
...,...,...,...,...,...
247734,0.0,0.0,0.0,5.0,0.0
247735,4.5,0.0,0.0,0.0,0.0
247738,0.0,0.0,4.0,0.0,0.0


In [13]:
movies_users.index[0]

4

In [14]:
matrix_mv_users = csr_matrix(movies_users.values)
X = movies_users.values
X


array([[0., 0., 4., 0., 0.],
       [0., 0., 0., 0., 3.],
       [0., 2., 0., 5., 0.],
       ...,
       [0., 0., 4., 0., 0.],
       [4., 0., 4., 5., 0.],
       [3., 0., 0., 0., 0.]])

# NearestNeighbors
# USer-based

In [15]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(metric = 'cosine', algorithm='brute', n_neighbors=20).fit(X)

x_in = np.array(inputMovies.rating.tolist())
x_in = x_in.reshape(1,5)
distances, indices = nbrs.kneighbors(x_in)
indices[0]

array([ 54553,  84868,  47136,  39577, 114232, 102216,  33500,  66479,
       106764,  15401,  52759, 102413,  24144,  33569,  20496,  70805,
         5312,  70728,  26909,  47058])

In [16]:
distances[0]

array([0.01087195, 0.0192767 , 0.0196157 , 0.02008813, 0.02102885,
       0.02142265, 0.02156425, 0.02203368, 0.02210581, 0.02272727,
       0.026823  , 0.02725309, 0.02736005, 0.02793849, 0.02808754,
       0.02900182, 0.02991996, 0.03016341, 0.03018177, 0.03030303])

In [17]:
np.shape(distances)

(1, 20)

In [18]:
df_t = pd.DataFrame()
df_t['similarityIndex'] = 1/distances[0]
df_t['userId'] = movies_users.index[indices[0]]
df_t.head()

,similarityIndex,userId
0,91.979841,116108
1,51.876086,181070
2,50.979584,100550
3,49.780641,84198
4,47.553728,243600


In [19]:
#!pip install fuzzywuzzy
#from fuzzywuzzy import process
#idx = process.extractOne('Toy Story',movies_df['title'])

#### The top x similar users to input user

Now let's get the top 100 users that are most similar to the input.


In [20]:
topUsers=df_t.sort_values(by='similarityIndex', ascending=False)[0:100]
topUsers.head()
len(topUsers)

20

In [21]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head(20)

,similarityIndex,userId,movieId,rating
0,91.979841,116108,1,2.5
1,91.979841,116108,2,3.5
2,91.979841,116108,6,4.5
3,91.979841,116108,17,4.0
4,91.979841,116108,19,0.5
5,91.979841,116108,21,3.0
6,91.979841,116108,29,4.0
7,91.979841,116108,32,3.0
8,91.979841,116108,34,4.0
9,91.979841,116108,36,3.0


In [22]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,91.979841,116108,1,2.5,229.949602
1,91.979841,116108,2,3.5,321.929443
2,91.979841,116108,6,4.5,413.909284
3,91.979841,116108,17,4.0,367.919364
4,91.979841,116108,19,0.5,45.989920


In [23]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,868.953036,2980.280065
2,868.953036,3267.994816
3,244.484828,789.794269
4,134.656728,368.874738
5,73.974531,239.975966


In [24]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.429737,1
2,3.760842,2
3,3.230443,3
4,2.739371,4
5,3.244035,5


Now let's sort it and see the top 20 movies that the algorithm recommended!


In [25]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
3545,5.0,3545
1280,5.0,1280
56003,5.0,56003
27011,5.0,27011
2506,5.0,2506


In [26]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(20)['movieId'].tolist())]

,movieId,title
211,213,Burnt by the Sun (Utomlyonnye solntsem) (1994)
557,562,Welcome to the Dollhouse (1995)
710,722,"Haunted World of Edward D. Wood Jr., The (1996)"
1252,1280,Raise the Red Lantern (Da hong deng long gao g...
1580,1635,"Ice Storm, The (1997)"
1603,1660,Eve's Bayou (1997)
1912,1995,Poltergeist II: The Other Side (1986)
2422,2506,"Other Sister, The (1999)"
2441,2525,Alligator (1980)
2466,2550,"Haunting, The (1963)"


### Advantages and Disadvantages of Collaborative Filtering

##### Advantages

-   Takes other user's ratings into consideration
-   Doesn't need to study or extract information from the recommended item
-   Adapts to the user's interests which might change over time

##### Disadvantages

-   Approximation function can be slow
-   There might be a low of amount of users to approximate
-   Privacy issues when trying to learn the user's preferences


<h2>Want to learn more?</h2>

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: <a href="https://www.ibm.com/analytics/spss-statistics-software">SPSS Modeler</a>

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at <a href="https://www.ibm.com/cloud/watson-studio">Watson Studio</a>


### Thank you for completing this lab!

## Author

Saeed Aghabozorgi

### Other Contributors

<a href="https://www.linkedin.com/in/joseph-s-50398b136/" target="_blank">Joseph Santarcangelo</a>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                 |
| ----------------- | ------- | ---------- | ---------------------------------- |
| 2020-11-03        | 2.1     | Lakshmi    | Updated URL of csv                 |
| 2020-08-27        | 2.0     | Lavanya    | Moved lab to course repo in GitLab |
|                   |         |            |                                    |
|                   |         |            |                                    |

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
